In [ ]:
from utils.dataset import HumanEvalDataset
data_root ='data/'
language = 'python'
dataset = HumanEvalDataset(data_root, language=language)


GENERATE CODE

In [11]:
import glob 
import json
import os
for file in glob.glob('human_eval\\open_ai_call\\output_*.jsonl'):
    source_file = file.replace('output_','')
    mapping_file = source_file +".mapping.json"
    data = list()
    if os.path.exists(mapping_file):
        with open(mapping_file) as mp_file:
            mapping = json.load(mp_file)
    else:
        mapping = dict()
    with open(file) as f:
        for l in f.readlines():
            obj = json.loads(l.strip())
            if obj['custom_id'].isnumeric():
                obj['custom_id'] = mapping[obj['custom_id']]
            data.append(obj)
    with open(file,'w+') as f:
        for obj in data:
            f.writelines(json.dumps(obj)+'\n')
    # print(file)
    # print(data[0])

**merge**

In [14]:
temps = [
    'human_eval/open_ai_call/output_mini_batch_*_java_control_dep.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_java_data_dep.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_java_method_name.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_java_output.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_java_summarize.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_python_control_dep.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_python_data_dep.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_python_method_name.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_python_output.jsonl',
'human_eval/open_ai_call/output_mini_batch_*_python_summarize.jsonl',
]
for temp in temps:
    out = temp.replace('*','final')
    data = list()
    for file in glob.glob(temp):
        with open(file) as f:
            for l in f.readlines():
                obj = json.loads(l.strip())
                data.append(obj)
    with open(out,'w+') as f:
        for obj in data:
            f.writelines(json.dumps(obj)+'\n')
    print(out)

human_eval/open_ai_call/output_mini_batch_final_java_control_dep.jsonl
human_eval/open_ai_call/output_mini_batch_final_java_data_dep.jsonl
human_eval/open_ai_call/output_mini_batch_final_java_method_name.jsonl
human_eval/open_ai_call/output_mini_batch_final_java_output.jsonl
human_eval/open_ai_call/output_mini_batch_final_java_summarize.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_control_dep.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_data_dep.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_method_name.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_output.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_summarize.jsonl


GERERATE DEPENDENCE

In [28]:
# xxx
import pandas as pd 
import json 
lang = 'java'
type_dep = 'control_dependence'

def parser_jsonl(file):
    result = list()
    with open(file) as f:
        for l in f.readlines():
            obj = json.loads(l.strip())
            res = obj['response']['body']['choices'][0]['message']['content']
            custom_id = obj['custom_id']
            task_id = custom_id.split("#####")[0]
            result.append({'task_id':task_id,'response':res})
    return result

# human_eval/open_ai_call/output_mini_batch_final_java_control_dep.jsonl
# human_eval/open_ai_call/output_mini_batch_final_java_data_dep.jsonl
# human_eval/open_ai_call/output_mini_batch_final_java_method_name.jsonl
# human_eval/open_ai_call/output_mini_batch_final_java_output.jsonl
# human_eval/open_ai_call/output_mini_batch_final_java_summarize.jsonl  
import pandas as pd
langs = ['java','python']
deps = ['data_dep','control_dep']     
for lang in langs:
    for dep in deps:
        file = f"human_eval/open_ai_call/output_mini_batch_final_{lang}_{dep}.jsonl"
        data = parser_jsonl(file)
        print(data[0])
        df = pd.DataFrame(data)
        df.to_csv(f'human_eval/{lang}/{dep}_{lang}.csv',index=False)



{'task_id': 'HumanEval_23_strlen', 'response': 'There are no pairs of statements in the given source code that have a data dependence relationship.'}
{'task_id': 'HumanEval_23_strlen', 'response': 'There are no pairs of statements in the given source code that have a control dependence relationship.'}
{'task_id': 'Python/0', 'response': '1. Line 5 depends on line 4\n2. Line 6 depends on line 4\n3. Line 6 depends on line 5\n4. Line 7 depends on line 6'}
{'task_id': 'Python/0', 'response': '1. Line 3 and Line 5\n2. Line 5 and Line 7'}


GENERATE NAME PRD

In [27]:
# xxx
import pandas as pd 
import json 
lang = 'java'

def parser(file):
    result = list()
    with open(file) as f:
        for l in f.readlines():
            obj = json.loads(l.strip())
            res = obj['response']['body']['choices'][0]['message']['content']
            custom_id = obj['custom_id'].split("#####")
            # assert len(custom_id) == 3            
            task_id,name,partition = custom_id[0],custom_id[1], custom_id[2]
            result.append({'task_id': task_id,'original_method':name,
                           'response': res, 'partition':partition})
    return result 

# human_eval/open_ai_call/output_mini_batch_final_java_method_name.jsonl
# human_eval/open_ai_call/output_mini_batch_final_java_output.jsonl
# human_eval/open_ai_call/output_mini_batch_final_java_summarize.jsonl 
langs = ['java','python']
for lang in langs:
    file = f"human_eval/open_ai_call/output_mini_batch_final_{lang}_method_name.jsonl"
    data = parser(file)
    df = pd.DataFrame(data)
    df.to_csv(f'human_eval/{lang}/method_name_prd_{lang}.csv',index=False)
    print(file)
            

human_eval/open_ai_call/output_mini_batch_final_java_method_name.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_method_name.jsonl


GENERATE SUMMARY

In [26]:
# xxx
import pandas as pd 
import json 
lang = 'java'

def parser(file):
    result = list()
    with open(file) as f:
        for l in f.readlines():
            obj = json.loads(l.strip())
            res = obj['response']['body']['choices'][0]['message']['content']
            custom_id = obj['custom_id'].split("#####")
            # assert len(custom_id) == 3      
            task_id,partition = custom_id[0],custom_id[1]
            result.append({'task_id': task_id, 'response': res, 'partition':partition})
    return result 
            
langs = ['java','python']
for lang in langs:
    file = f"human_eval/open_ai_call/output_mini_batch_final_{lang}_summarize.jsonl"
    data = parser(file)
    df = pd.DataFrame(data)
    df.to_csv(f'human_eval/{lang}/summarize_{lang}.csv',index=False)
    print(file)

human_eval/open_ai_call/output_mini_batch_final_java_summarize.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_summarize.jsonl


GENERATE OUTPUT

In [25]:
# xxx
import pandas as pd 
import json 
lang = 'java'

def parser(file):
    result = list()
    with open(file) as f:
        for l in f.readlines():
            obj = json.loads(l.strip())
            res = obj['response']['body']['choices'][0]['message']['content']
            custom_id = obj['custom_id'].split("#####")
            # assert len(custom_id) == 3      
            task_id,test_input, expected_output, partition = custom_id[0],custom_id[1],custom_id[2],custom_id[3]
            result.append({'task_id': task_id, 'response': res, 
                           'partition':partition,
                           'test input': test_input,
                           "expected output": expected_output})
    return result 
            
langs = ['java','python']
for lang in langs:
    file = f"human_eval/open_ai_call/output_mini_batch_final_{lang}_output.jsonl"
    data = parser(file)
    df = pd.DataFrame(data)
    df.to_csv(f'human_eval/{lang}/output_{lang}.csv',index=False)
    print(file)

human_eval/open_ai_call/output_mini_batch_final_java_output.jsonl
human_eval/open_ai_call/output_mini_batch_final_python_output.jsonl


DRIVE